# 🎬 Project 3 – Part 3: ETL to PostgreSQL

**Author:** Joseph Tulani Aytch  
**Last Updated:** Aug 2025  

---

## 🎯 Objective
Load cleaned movie datasets from Parts 0–2, transform them for relational storage,  
and create a normalized PostgreSQL database with these tables:
- `title_basics`
- `title_ratings`
- `title_genres`
- `genres`
- `tmdb_data`

---

## 🛠 Workflow
1. **Load cleaned CSVs** from the `Data/` folder (repo‑stored for reproducibility)  
2. **Normalize** genres into lookup/join tables  
3. **Define schemas** with SQLAlchemy  
4. **Save tables** to PostgreSQL  
5. **Verify** with sample queries  

---

## 📌 Deliverables
- PostgreSQL database `movies`
- All tables above populated from cleaned data
- Notebook documenting ETL steps

---

## ⚠ Notes
- Requires a local PostgreSQL instance and valid credentials in the connection URI.
- Only uses relative paths — runs anywhere with a cloned repo.


## Setup Notes

This project requires the following Python packages:

- **SQLAlchemy** → ORM and database connection handling
- **psycopg2-binary** → PostgreSQL driver
- **yelpapi** → Wrapper for Yelp Fusion API requests
- **pandas** → Data handling and analysis
- **requests** → HTTP requests (Yelp API calls if not using the wrapper)

Install them via:

```bash
pip install SQLAlchemy psycopg2-binary yelpapi pandas requests


In [12]:
# pip install yelpapi

In [7]:
# pip install psycopg2-binary

In [2]:
## 1. Imports & Paths
from pathlib import Path
import pandas as pd
from sqlalchemy import create_engine

# Relative data directory
data_dir = Path("Data")


In [3]:
## 2. Load Cleaned CSVs
basics = pd.read_csv(data_dir / "final_basics.csv.gz", compression="gzip")
ratings = pd.read_csv(data_dir / "final_ratings.csv.gz", compression="gzip")
results = pd.read_csv(data_dir / "tmdb_results_combined.csv.gz", compression="gzip")


In [4]:
## 3. Normalize Genres (Lookup Table + Join Table)
# Split genres into list
basics['genres_list'] = basics['genres'].str.split(',')

# Explode into separate rows
title_genres = basics[['tconst', 'genres_list']].explode('genres_list').dropna()

# Deduplicate for genres table
genres = pd.DataFrame({'genre': sorted(title_genres['genres_list'].unique())}).reset_index(drop=True)
genres['genre_id'] = genres.index + 1

# Map genre names to IDs
genre_map = dict(zip(genres['genre'], genres['genre_id']))
title_genres['genre_id'] = title_genres['genres_list'].map(genre_map)
title_genres = title_genres[['tconst', 'genre_id']]


In [13]:
from sqlalchemy import create_engine, text, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

# --- Connection settings ---
pg_user = "postgres"
pg_pass = "password"
pg_host = "localhost"
pg_port = "5432"
db_name = "movies"

# 1️⃣ Connect to default database
default_engine = create_engine(
    f"postgresql+psycopg2://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/postgres"
)

# 2️⃣ Create the target database if missing
with default_engine.connect() as conn:
    conn.execution_options(isolation_level="AUTOCOMMIT")
    exists = conn.execute(
        text("SELECT 1 FROM pg_database WHERE datname=:name"),
        {"name": db_name}
    ).scalar()
    if not exists:
        conn.execute(text(f"CREATE DATABASE {db_name}"))
        print(f"✅ Database '{db_name}' created")
    else:
        print(f"⚡ Database '{db_name}' already exists")

# 3️⃣ Connect to the new database
engine = create_engine(
    f"postgresql+psycopg2://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{db_name}"
)

# 4️⃣ Define table schema
Base = declarative_base()

class Movies(Base):
    __tablename__ = "movies"
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    genre = Column(String)
    rating = Column(Float)

# 5️⃣ Create the table
Base.metadata.create_all(engine)
print("🎬 Table(s) are ready for data inserts!")


✅ Database 'movies' created
🎬 Table(s) are ready for data inserts!


C:\Users\taytch\AppData\Local\Temp\ipykernel_3876\2050346806.py:35: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [14]:
## 5. Write Tables to PostgreSQL
basics.to_sql("title_basics", engine, if_exists="replace", index=False)
ratings.to_sql("title_ratings", engine, if_exists="replace", index=False)
genres.to_sql("genres", engine, if_exists="replace", index=False)
title_genres.to_sql("title_genres", engine, if_exists="replace", index=False)
results.to_sql("tmdb_data", engine, if_exists="replace", index=False)

586

In [15]:
## 6. Verify Tables
pd.read_sql("""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema='public';
""", engine)


,table_name
0,movies
1,title_basics
2,title_ratings
3,genres
4,title_genres
5,tmdb_data


In [20]:
# Merge basics + ratings into a unified table
movies_df = basics.merge(
    ratings,
    on="tconst",
    how="left"
).merge(
    title_genres.merge(genres, on="genre_id"),
    on="tconst",
    how="left"
)

# Rename columns to match Movies schema
movies_df = movies_df.rename(columns={
    "primaryTitle": "title",
    "genre": "genre",
    "averageRating": "rating"
})[["title", "genre", "rating"]]

# Optional: drop duplicates if the genre join made multiples
movies_df = movies_df.drop_duplicates()

# Load into the movies table
movies_df.to_sql("movies", engine, if_exists="replace", index=False)

print(f"✅ Populated movies table with {len(movies_df)} rows")


✅ Populated movies table with 156835 rows


## 📊 SQL Showcase – Analytical Queries

Below are example queries run against our `movies` database to demonstrate SQL capabilities, multi‑table joins, aggregations, and window functions.



### Top 10 Highest‑Rated Movies

**Question:** Which 10 movies have the highest average ratings?  
**Why it matters:** Quickly surfaces top‑performing titles for recommendations or promotion.

In [28]:
query = text("""
    SELECT title, genre, rating
    FROM movies
    WHERE rating IS NOT NULL
    ORDER BY rating DESC
    LIMIT 10;
""")

pd.read_sql(query, engine)

,title,genre,rating
0,Come Back Home,Drama,10.0
1,Zombies in the Antique Shop,Horror,10.0
2,Sway,Comedy,10.0
3,King B.'s Intimate Friends,Romance,10.0
4,King B.'s TPD Experience,Drama,10.0
5,Zombies in the Antique Shop,Comedy,10.0
6,True Fighter,Drama,10.0
7,Major,Drama,10.0
8,Don: Plain & Tall,Comedy,10.0
9,Afternoon Is Night,Comedy,10.0


### Number of Titles per Genre

- Question: Which genres have the largest number of titles? 
- Why it matters: Useful for understanding content inventory and category weight.

In [30]:
query = text("""
    SELECT genre, COUNT(*) AS num_titles
    FROM movies
    GROUP BY genre
    ORDER BY num_titles DESC;
""")

pd.read_sql(query, engine)


,genre,num_titles
0,Drama,44360
1,Comedy,24729
2,Thriller,12620
3,Horror,12079
4,Action,10622
5,Romance,9965
6,Crime,7860
7,Adventure,5631
8,Mystery,5283
9,Family,3934


## Top 3 Movies per Genre by Rating

- Question: Within each genre, what are the top 3 movies by rating?
- Why it matters: Demonstrates window functions and ranking – key analytical SQL skills.

In [29]:
query = text("""
    SELECT genre, title, rating
    FROM (
        SELECT 
            m.genre,
            m.title,
            m.rating,
            ROW_NUMBER() OVER (
                PARTITION BY m.genre
                ORDER BY m.rating DESC
            ) AS rank_within_genre
        FROM movies m
        WHERE m.rating IS NOT NULL
    ) ranked
    WHERE rank_within_genre <= 3
    ORDER BY genre, rank_within_genre;
""")

pd.read_sql(query, engine)


,genre,title,rating
0,Action,Masand,9.9
1,Action,Thorn in the Center of the Heart (Sin Lai Guh ...,9.7
2,Action,The Return of Uncle Benon,9.6
3,Adult,Maid to Kill,8.8
4,Adult,Undercover,8.6
...,...,...,...
71,War,Ek Sainik - The Tale of a warrior,9.2
72,War,"O Jehovah, ... I Trust in You",8.8
73,Western,Cold Cross,9.5
74,Western,The Never Daunted,8.9
